In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os
import torchvision
import torchvision.transforms as transforms 

In [7]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes,planes,kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Identity()
        if stride != 1:
            self.shortcut = nn.Sequential(
                            nn.Conv2d(in_planes, planes, kernel_size = 1, stride = stride, bias = False),
                            nn.BatchNorm2d(planes)
                            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)

        return out

In [8]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3,64,kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride = 1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride = 2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride = 2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride = 2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1) # out.size(0) = out.reshape(0)
        out = self.linear(out)

        return out

In [9]:
train = transforms.Compose([
    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root= './data', train = True, download = True, transform = train)
test_dataset = torchvision.datasets.CIFAR10(root= './data', train = False, download = True, transform = test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, shuffle = True, num_workers = 4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = False, num_workers = 4)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

In [17]:
device = 'cuda'
cudnn.benchmark = True

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)

learning_rate = 0.1
file_name = 'resnet18_cifar10.pth'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum = 0.9, weight_decay = 0.0002)

In [18]:
def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in  enumerate(train_loader):
        inputs= inputs.to(device)
        targets = targets.to(device)

        outputs = net(inputs)

        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)

In [19]:
def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

In [20]:
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [21]:
for epoch in range(0, 20):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1015625
Current benign train loss: 2.3281350135803223

Current batch: 100
Current benign train accuracy: 0.3515625
Current benign train loss: 1.8828078508377075

Current batch: 200
Current benign train accuracy: 0.296875
Current benign train loss: 1.8152875900268555

Current batch: 300
Current benign train accuracy: 0.359375
Current benign train loss: 1.7569916248321533

Total benign train accuarcy: 30.184
Total benign train loss: 761.4594428539276

[ Test epoch: 0 ]

Test accuarcy: 39.0
Test average loss: 0.016725183475017547
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.4140625
Current benign train loss: 1.5157055854797363

Current batch: 100
Current benign train accuracy: 0.4140625
Current benign train loss: 1.5088170766830444

Current batch: 200
Current benign train accuracy: 0.453125
Current benign train loss: 1.5140979290008545

Current batch: 300
Current benign train accu

In [22]:
def test_model_on_test_set(model, test_loader):
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

def load_model_with_prefix(model, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict({k.replace('module.', ''): v for k, v in checkpoint['net'].items()})

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet18()
model = model.to(device)

checkpoint_path = './checkpoint/' + file_name
if os.path.exists(checkpoint_path):
    load_model_with_prefix(model, checkpoint_path)
    print('Model weights loaded successfully!')
else:
    print('Model weights not found.')

test_model_on_test_set(model, test_loader)

C:\Users\DenisDrobyshev\AppData\Local\Temp\ipykernel_25400\1069915622.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


Model weights loaded successfully!
Accuracy on test set: 84.02%
